<a href="https://colab.research.google.com/github/gibsonx/CE888/blob/master/Federated_Learning_DockerPy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Modified by Gibson Xue on July 25, 2021
!pip uninstall --yes tensorflow tensorboard tb-nightly
!pip install --quiet tensorflow==2.5.0
!pip install --quiet tensorflow-federated==0.19.0
!pip install --quiet nest-asyncio
!pip install --quiet tensorboard  # or tensorboard, but not both
!pip install keras
import tensorflow as tf

import nest_asyncio
nest_asyncio.apply()
print(tf.__version__)

from tensorflow_federated.python.simulation import FileCheckpointManager
import collections
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import pandas as pd

Uninstalling tensorflow-2.5.0:
  Successfully uninstalled tensorflow-2.5.0
Uninstalling tensorboard-2.5.0:
  Successfully uninstalled tensorboard-2.5.0
2.5.0


In [3]:
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

In [4]:
from google.colab import drive
drive.mount('/content/drive')
!cp /content/drive/MyDrive/train.csv /content

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# example_dataset = emnist_train.create_tf_dataset_for_client(
#     emnist_train.client_ids[0])
# for i in example_dataset.take(1):
#   print(i)
data = pd.read_csv('train.csv')
train_data = data.head(37800)
val_data = data.tail(4200)
train_labels = train_data.pop('label')
val_labels = val_data.pop('label')
tf_train_data = tf.data.Dataset.from_tensor_slices((train_data.values, train_labels.values))
tf_val_data = tf.data.Dataset.from_tensor_slices((val_data.values, val_labels.values))

# print(type(example_dataset))

# import tensorflow_datasets as tfds

# np.savetxt("foo.csv", a, delimiter=",")
# import numpy as geek

# list_train = []
# len(emnist_train.client_ids)
# for i in range(0,1):
#   client_dataset = emnist_train.create_tf_dataset_for_client(
#       emnist_train.client_ids[i])
#   print(client_dataset.element_spec)
#   for example in client_dataset:
#     pixels = example['pixels'].numpy()
#     label = geek.array_str(example['pixels'].numpy())
#     list_train.append({'pixels': pixels, 'label': label, 'client_id': i, 'is_checked': 0 })
    # # t = geek.array(g)
    # print(pixels)
    # break

# list_train
# df = pd.DataFrame(list_train)
# df.to_csv('data.csv',index=False)
# compression_opts = dict(method='zip',archive_name='data.csv')
# df.to_csv(data.csv, index=False,
#           compression=compression_opts)


In [6]:
# df = pd.read_csv('data.csv')
# dataset = tf.data.experimental.CsvDataset('data.csv',[tf.int32],select_cols=[1])

# d = tf.convert_to_tensor(g,dtype=tf.float32)
# d
# for i in dataset:
#   print(i)
# number_column = df.loc[:,'pixels'].apply(lambda x : tf.convert_to_tensor(x, dtype=tf.float32))
# number_column
# df = tf.data.experimental.make_csv_dataset('data.csv',batch_size=2)
# for i in number_column:
#   print(i)
#   break
# example_dataset = emnist_train.create_tf_dataset_for_client(
#     emnist_train.client_ids[0])

# example_dataset

# preprocessed_example_dataset = preprocess(df.to_numpy())

# preprocessed_example_dataset = preprocess(example_dataset)

In [7]:
NUM_CLIENTS = 10
NUM_EPOCHS = 5
BATCH_SIZE = 20
SHUFFLE_BUFFER = 100
PREFETCH_BUFFER = 10

def preprocess(dataset):

  def batch_format_fn(element):
    """Flatten a batch `pixels` and return the features as an `OrderedDict`."""
    return collections.OrderedDict(
        x=tf.reshape(element['pixels'], [-1, 784]),
        y=tf.reshape(element['label'], [-1, 1]))

  return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(
      BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)

In [8]:
def preprocess_image(image, label):
    image = tf.reshape(image, [28, 28, 1])
    image = tf.cast(image, tf.float32) / 255.
    
    return image, label

tf_train_data = tf_train_data.map(
    preprocess_image, 
    num_parallel_calls=tf.data.experimental.AUTOTUNE
)

tf_val_data = tf_val_data.map(
    preprocess_image, 
    num_parallel_calls=tf.data.experimental.AUTOTUNE
)

def pipeline(tf_data):
    tf_data = tf_data.shuffle(100)
    tf_data = tf_data.batch(32)
    tf_data = tf_data.prefetch(tf.data.experimental.AUTOTUNE)
    
    return tf_data

tf_train_data = pipeline(tf_train_data)
tf_val_data = pipeline(tf_val_data)

In [12]:
tf_train_data

list_1 = []
list_2 = []
for example in tf_train_data.take(1):
  list_1.append(example[0].numpy())
  list_2.append(example[1].numpy())

# collections.OrderedDict(x=list_1,y=list_2)
dataset = tf.data.Dataset.from_tensors(collections.OrderedDict(pixels=list_1,label=list_2))
preprocessed_example_dataset = preprocess(dataset)

In [13]:
def make_federated_data(client_data, client_ids):
  return [
      preprocess(client_data.create_tf_dataset_for_client(x))
      for x in client_ids
  ]
#tf.keras.layers.InputLayer(input_shape=(784,)),
def create_keras_model():
  return tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(784,)),
      tf.keras.layers.Dense(10, kernel_initializer='zeros'),
      tf.keras.layers.Softmax(),
  ])

def model_fn():
  # We _must_ create a new model here, and _not_ capture it from an external
  # scope. TFF will call this within different graph contexts.
  keras_model = create_keras_model()
  return tff.learning.from_keras_model(
      keras_model,
      input_spec=preprocessed_example_dataset.element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [14]:
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))

In [15]:
client_dataset = emnist_train.create_tf_dataset_for_client(
      emnist_train.client_ids[0])

client_dataset_proce = preprocess(client_dataset)
dataset_apply = make_federated_data(emnist_train,emnist_train.client_ids[0:1])
dataset_apply

[<PrefetchDataset shapes: OrderedDict([(x, (None, 784)), (y, (None, 1))]), types: OrderedDict([(x, tf.float32), (y, tf.int32)])>]

In [23]:
state = iterative_process.initialize()

# state

# dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(
#       BATCH_SIZE).map(g).prefetch(PREFETCH_BUFFER)
def make_federated(client_data):
  return [
      preprocess(dataset)
      for x in client_data
  ]

x = make_federated(dataset)
state, metrics = iterative_process.next(state, x)
# preprocessed_example_dataset = preprocess(example_dataset)

# sample_batch = tf.nest.map_structure(lambda x: x.numpy(),
#                                      next(iter(preprocessed_example_dataset)))

# sample_batch
# x = preprocess(dataset)
# dataset
# state, metrics = iterative_process.next(state, x)
# metrics
x

TypeError: ignored

In [22]:
metrics

OrderedDict([('broadcast', ()),
             ('aggregation',
              OrderedDict([('mean_value', ()), ('mean_weight', ())])),
             ('train',
              OrderedDict([('sparse_categorical_accuracy', 0.15625),
                           ('loss', 2.3025854)])),
             ('stat', OrderedDict([('num_examples', 160)]))])